# Energy Monitor

## Initializations

In [ ]:
import logging
from pathlib import Path
import polars as pl
import altair as alt

from analyze.loader import DeviceDataSource
from analyze.logger import POLAR_ANALYZER_LOGGER
from analyze.model import PolarDeviceData
from analyze.common import PHASE_COLUMNS, Phase
from config import config

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(threadName)s - %(levelname)s - %(name)s - %(message)s")
POLAR_ANALYZER_LOGGER.setLevel(logging.INFO)
logger = POLAR_ANALYZER_LOGGER.getChild("notebook")

## Load data

In [ ]:
data = PolarDeviceData.load([DeviceDataSource(f.dir, f.device) for f in config.files])

## Energy by Day of the Week

In [ ]:
week_day = data.total_energy_by_day_of_week()
week_day = week_day.with_columns(pl.concat_str([pl.col("device"), pl.col("phase")], separator="_").alias("device_phase"))
week_day = week_day.collect()
week_day.head(3)

In [ ]:
week_day_chart = (
    alt.Chart(week_day)
    .mark_bar(size=15, tooltip=True)
    .encode(
        x=alt.X(field="day_of_week", type="nominal").title("Day of week").sort({"field": "day_of_week_num"}),
        y=alt.Y(field="total_act_energy_kwh", type="quantitative").title("Energy [kWh]"),
        color=alt.Color(field="device_phase", type="nominal").title("Device / Phase"),
        order=alt.Order(field="device_phase", type="nominal"),
    )
    .properties(title="Energy by day of week", width=250, height=400)
)
week_day_chart

## Energy by Month

In [ ]:
month = data.total_energy(every="1mo")
month = month.with_columns(pl.concat_str([pl.col("device"), pl.col("phase")], separator="_").alias("device_phase"))
month = month.collect()
month.head(3)

In [ ]:
chart = (
    alt.Chart(month)
    .mark_bar(size=20, tooltip=True)
    .encode(
        x=alt.X("date", type="temporal").title("Date"),
        y=alt.Y("total_act_energy_kwh", type="quantitative").title("Energy [kWh]"),
        color=alt.Color("device_phase", type="nominal").title("Device / Phase"),
    )
    .properties(title="Energy by month", width=400, height=400)
)
chart

## Energy by Week

In [ ]:
week = data.total_energy(every="1w")
week = week.with_columns(pl.concat_str([pl.col("device"), pl.col("phase")], separator="_").alias("device_phase"))
week = week.collect()
week.head(3)

In [ ]:
week_chart = (
    alt.Chart(week)
    .mark_bar(size=10, tooltip=True)
    .encode(
        x=alt.X("date", type="temporal").title("Date"),
        y=alt.Y("total_act_energy_kwh", type="quantitative").title("Energy [kWh]"),
        color=alt.Color("device_phase", type="nominal").title("Device / Phase"),
    )
    .properties(title="Energy by week", width=600, height=400)
)
week_chart
